In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import seaborn as sns

players = pd.read_csv('./Dataset/players.csv')
teams = pd.read_csv('./Dataset/teams.csv')
players_teams = pd.read_csv('./Dataset/players_teams.csv')
awards_players = pd.read_csv('./Dataset/awards_players.csv')
coaches = pd.read_csv('./Dataset/coaches.csv')
series_post = pd.read_csv('./Dataset/series_post.csv')
teams_post = pd.read_csv('./Dataset/teams_post.csv')



### Awards_players


- corrected name of a prize
- drop lgID
- calculate the number of prizes that each player has (having in consideration the year)
- drop award (name)

In [2]:
awards_players['award'] = awards_players['award'].replace("Kim Perrot Sportsmanship", "Kim Perrot Sportsmanship Award")
awards_players = awards_players.drop(columns=['lgID'], axis=1)

# Sort the DataFrame by playerID and year
awards_players.sort_values(by=["playerID", "year"], inplace=True)

awards_players['num_prizes'] = awards_players.groupby('playerID').cumcount() + 1

awards_players = awards_players.drop(columns=['award'], axis=1)

awards_players.to_csv('modified_awards.csv', index=False)


### Players

- drop college, collegeOther, pos and lastseason
- remove who has height = 0
- replace height error (9) with the average
- replace weight error (0) with the average


In [3]:
players = players.drop('college', axis=1)
players = players.drop('pos', axis=1)
players = players.drop('collegeOther', axis=1)
# Have only 0s
players = players.drop('lastseason', axis=1)

players = players[players['height'] > 0]

# Calculate the mean of non-zero heights
mean_height = players[(players['height'] != 0) | (players['height'] != 9)]['height'].mean()
# Replace 9 with the mean height
players['height'] = players['height'].replace(9, mean_height)

# Calculate the mean of non-zero weights
mean_weight = players[players['weight'] != 0]['weight'].mean()
players['weight'] = players['weight'].replace(0, mean_weight)



- add a column called Age, considering the of the season. Calculated through the merge between two datasets
- drop birthdate and deathdate

CORRECT WHEN DOESNT HAVE PRZIEEEE

In [4]:
def calculate_age(row):
    birth_year = int(row['birthDate'][:4])
    age = 2000 + row['year'] - birth_year
    return age


merged_data = players.merge(players_teams, left_on='bioID', right_on='playerID')

merged_data = merged_data.merge(awards_players, left_on=['bioID', 'year'], right_on=['playerID', 'year'], how='inner')

# Apply the function to each row
merged_data['age'] = merged_data.apply(calculate_age, axis=1)

merged_data = merged_data.drop('birthDate', axis=1)
merged_data = merged_data.drop('deathDate', axis=1)



In [5]:
# Save the modified DataFrame to a CSV file
players.to_csv('modified_players.csv', index=False)

### Teams

- drop seeded, confID, divID, franchID and arena
- corrected error: one season it was registered two winners

In [6]:

teams = teams.drop(columns=['seeded'], axis=1)
teams = teams.drop(columns=['confID'], axis=1)
teams = teams.drop(columns=['divID'], axis=1)
teams = teams.drop(columns=['franchID'], axis=1)
teams = teams.drop(columns=['arena'], axis=1)


# In the year 6, a victory was incorrectly recorded.
teams.loc[(teams['year'] == 6) & (teams['tmID'] == 'CON'), 'finals'] = 'L'



 calcular o numero de premios que cada equipa tem no total (de acordo com o numero de premio que cada jogadora tem) de acordo com o ano

In [7]:

playerAndTeams = merged_data.merge(teams, on=["tmID", "year"])

# Filter for teams that won the finals
#winners_data = playerAndTeams[playerAndTeams['finals'] == 'W']

# Group by team and year, summing the number of prizes
team_prizes = playerAndTeams.groupby(['tmID', 'year'])['num_prizes'].sum().reset_index()

# Fill NaN values with 0 for teams with no prizes
team_prizes['num_prizes'].fillna(0, inplace=True)

# Sort the result by year
team_prizes.sort_values(by='year', ascending=True, inplace=True)
print(team_prizes)

teams = teams.merge(team_prizes, on=["tmID", "year"])


   tmID  year  num_prizes
10  HOU     1           5
2   CLE     1           1
34  NYL     1           1
30  MIN     1           1
29  MIA     2           1
23  LAS     2           6
42  POR     2           1
35  NYL     2           1
11  HOU     2           1
53  WAS     3           1
16  IND     3           1
24  LAS     3           9
37  PHO     3           1
12  HOU     3           7
13  HOU     4           6
43  SAC     4           1
6   DET     4           2
48  SEA     4           1
25  LAS     4           1
49  SEA     5           2
26  LAS     5          13
31  MIN     5           1
17  IND     5           1
3   CON     5           1
38  PHO     5           1
18  IND     6           2
14  HOU     6          13
44  SAC     6           1
54  WAS     6           1
4   CON     6           1
45  SAC     7           3
39  PHO     7           2
27  LAS     7          18
50  SEA     7           3
19  IND     7           7
15  HOU     7          13
32  MIN     7           1
7   DET     

In [8]:
teams.to_csv('modified_teams.csv', index=False)

merged_data.to_csv('merged_data.csv', index=False)

